# Modul 6 - Projektaufgabe
## Ugly is better than nothing - Generieren von Titelaufnahmen aus den METS/MODS-Daten digitalisierter Handschriften zur Übernahme in den K10plus und Nachnutzung im lokalen Nachweissystem ACTApro
### Ulrike Mehringer, 2023

In [1]:
import urllib.request
import xml.etree.ElementTree as ET
import datetime
import re
import os

## Personen: von GND zu PPN

In [2]:
# Funktion gnd_ppn: Personennormsatz-ID (PPN) über die GND-Nummer im K10Plus abrufen

#gnd_nr = "118655477"
def gnd_ppn(gnd_nr):
    
## Normsatz aus K10Plus auslesen
    url = "http://sru.k10plus.de/opac-de-627!rec=2?&operation=searchRetrieve&query=pica.nid="+gnd_nr+"&maximumRecords=1&recordSchema=mods"
    person_xml = "../data/person.xml"
    urllib.request.urlretrieve(url,person_xml)
    
## XML parsen
    person_tree = ET.parse(person_xml)
    person_root = person_tree.getroot()
    
## K10Plus-PPN aus person_xml auslesen
    rec_id = person_root.find('.//{http://www.loc.gov/mods/v3}recordIdentifier')
    
## Variable für die Textausgabe
    ppn = rec_id.text
    return(ppn)

    

## METS/MODS aus OpenDigi

In [3]:
# METS-Daten des OpenDigi-Projekts einlesen
#od_projekt = "Mh335"
od_projekt = "Mc339"
#od_projekt = "Mn1-245"
#od_projekt = "Mn1"
url = "https://idb.ub.uni-tuebingen.de/opendigi/"+od_projekt+"/mets"
od_xml = "../data/"+od_projekt+".xml"
urllib.request.urlretrieve(url,od_xml)

# Zieldatei anlegen
if os.path.exists("../data/"+od_projekt+".txt"):
    os.remove("../data/"+od_projekt+".txt")
datei = open("../data/"+od_projekt+".txt",'a')

# XML parsen
od_tree = ET.parse(od_xml)
od_root = od_tree.getroot()

In [4]:
# Bibliografische Gattung und Status, Feld 0500
od_x = od_root.find(".//{http://www.loc.gov/METS/}structMap")
print(od_x.attrib)
if od_x.attrib['TYPE']=='PHYSICAL':
    od_struc = od_root.findall(".//{http://www.loc.gov/METS/}div")
    for i in range(0,len(od_struc)):
        if od_struc[i].attrib["TYPE"]=="multivolume_work":
            od_logi = "Ofu"
            break
        else:
            od_logi = "Oau"
        ### hier Funktion einfügen: die url der GA auslesen, GA einlesen, schaun ob PPN-Digi vorhanden: 
        ### ja: ppn merken für 4160
        ### nein: ist blöd
else:
    od_logi = ("Ocu")
        
print(od_logi)
datei.write('0500 '+od_logi)

{'TYPE': 'PHYSICAL'}
Oau


8

In [5]:
# Entstehungsdatum, Feld 1100, 1100 $n
od_date = od_root.findall(".//{http://www.loc.gov/mods/v3}dateIssued")
# od_date ist eine Liste
od_date_norm = od_date[0].text
od_date_subm = "["+od_date[1].text+"]"
if re.search(r"[oO]. *[dDJ].",od_date_subm):
    od_date_norm = "[1XXX]"
    od_date_subm = "[Entstehungdatum nicht ermittelbar]"
print(od_date_norm,od_date_subm)
datei.write('\n1100 '+od_date_norm+'$n'+od_date_subm)

1393 [um 1393]


21

In [6]:
# Jahr der Digitalisierung, Feld 1109
today = datetime.date.today()
today_year = today.strftime("%Y")
print(today_year)
datei.write('\n1109 '+today_year)

2023


10

In [7]:
# Sprache, Feld 1500
od_lang = od_root.find(".//{http://www.loc.gov/mods/v3}languageTerm").text
if re.search(r"; *",od_lang):
    od_lang = str_replace("; ","$a",od_lang)
print(od_lang)
datei.write('\n1500 '+od_lang)

lat


9

In [8]:
# PIDs: URN und DOI, Felder 2050, 2051 und 4950
od_pid = od_root.findall(".//{http://www.loc.gov/mods/v3}identifier")
for i in range(0,len(od_pid)):
    if od_pid[i].attrib["type"]=="urn":
        od_urn = od_pid[i].text
        datei.write('\n2050 '+od_urn)
        datei.write('\n4950 http://nbn-resolving.de/'+od_urn+'$xR$3Volltext$534')
    elif od_pid[i].attrib["type"]=="doi":
        od_doi = od_pid[i].text
        datei.write('\n2051 '+od_doi)
        datei.write('\n4950 https://doi.org/'+od_doi+'$xR')
  

In [9]:
# Personen, Feld 3000, 3010

# Rollenkürzel Extension
role_ext = {'aut':'VerfasserIn',
            'rcp':'AdressatIn',
            'ill':'IllustratorIn',
            'oth':'Sonstigeyyy',
            'asn':'Sonstigexxxx'}
print(role_ext['aut'])

# alle Personnamen
od_name = od_root.findall(".//{http://www.loc.gov/mods/v3}name")

for i in range(0,len(od_name)):

    # Rolle 
    od_name_role = od_root.findall('.//{http://www.loc.gov/mods/v3}roleTerm')[i].text
    
    # Namen mit/ohne GND-Nummer, persönliche Namen, fingierte Rolle [Schreiber]
    y = str(od_name[i].attrib)
    if 'valueURI' in y:
        gnd_url = od_name[i].attrib['valueURI']
        gnd_nr = str(re.findall(r'/gnd/(.*)',gnd_url))
        name_ppn = gnd_ppn(gnd_nr)
        datei.write('\n3010 !'+name_ppn+'!$B'+role_ext[od_name_role]+'$4'+od_name_role)
        print(name_ppn,od_name_role)
    else:
        od_name_display = od_root.findall(".//{http://www.loc.gov/mods/v3}displayForm")[i].text
        # persönliche und moderne Namen
        if ',' in od_name_display:
            datei.write('\n3010 '+od_name_display+'$B'+role_ext[od_name_role]+'$4'+od_name_role)
        else:
            datei.write('\n3010 $P'+od_name_display+'$B'+role_ext[od_name_role]+'$4'+od_name_role)
            
        print(od_name_display,od_name_role)
    
        
    #für alle die Funktion auslesen hier und eckige Klammern rausschmeißen und bei asn bleiben.
    # Schreiber in 4046
    
#alle bekommen 3010

print(name_ppn) 



VerfasserIn
080046096 aut
Hieronymus Angeli de Esslingen [Schreiber] asn
Meier de Wasszneck, Ulrich [Schreiber] asn
080046096


In [10]:
# Signatur, Feld 4000 und 4065 $a
od_shelf = od_root.find(".//{http://www.loc.gov/mods/v3}shelfLocator").text
print(od_shelf)
datei.write('\n4065 XA-DE$cUB Tübingen$a'+od_shelf)

Mc 339


32

In [11]:
# Titel, Feld 4000
od_title = od_root.find(".//{http://www.loc.gov/mods/v3}title").text
print(od_title)
datei.write('\n4000 '+od_title+" - Tübingen, Universitätsbibliothek, "+od_shelf)

Novella super quarto libro decretalium Johannis Andreae


104

In [12]:
# Entstehungsort, Feld 4046 (Feld 4040?)
od_place = od_root.find(".//{http://www.loc.gov/mods/v3}placeTerm").text
print(od_place)
if re.search(r"[oO]. *[oO.]",od_place):
    od_place = "[Entstehungsort nicht ermittelbar]"
print(od_place) 
datei.write('\n4046 '+od_place+'$h'+od_date_subm)

o.O.
[Entstehungsort nicht ermittelbar]


51

In [13]:
# Festtext an die Ausgabedatei anhängen
datei.write('\n0501 Text$btxt')
datei.write('\n0502 Computermedien$bc')
datei.write('\n0503 Online-Ressource$bcr')
datei.write('\n1101 crxcn---apaup')
datei.write('\n1131 !10457187X!')
# datei.write('\n1505 $erda')
datei.write('\n4022 Online-Ausgabe')
datei.write('\n4048 Tübingen$nUniversitätsbibliothek')
datei.write('\n4201 Kurzaufnahme einer Handschrift')
datei.write('\n4233 $aaa$c2023$5DE-21')
datei.close()

# Notizen

In [14]:
# Nur als Gedächtnisstütze
od_data = od_root.find(".//{http://www.loc.gov/METS/}xmlData/*")
#type(od_data) -- xml.etree.ElementTree.Element
for child in od_data:
    print(child.tag,child.attrib)

{http://www.loc.gov/mods/v3}titleInfo {}
{http://www.loc.gov/mods/v3}identifier {'type': 'urn'}
{http://www.loc.gov/mods/v3}recordInfo {}
{http://www.loc.gov/mods/v3}identifier {'type': 'doi'}
{http://www.loc.gov/mods/v3}location {}
{http://www.loc.gov/mods/v3}language {}
{http://www.loc.gov/mods/v3}name {'authority': 'gnd', 'authorityURI': 'http://d-nb.info/gnd/', 'type': 'personal', 'valueURI': 'http://d-nb.info/gnd/119244071'}
{http://www.loc.gov/mods/v3}name {'type': 'personal'}
{http://www.loc.gov/mods/v3}name {'type': 'personal'}
{http://www.loc.gov/mods/v3}originInfo {}
